In [ ]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
import os
import time

In [ ]:
database_configuration = open(os.getcwd()+r'\db.cfg').read().split(' ')
engine = create_engine('mysql+mysqlconnector://{}:{}@{}:{}/{}'.format(*database_configuration))

In [ ]:
articulos_query = open(os.getcwd()+r'\articulos.sql').read()
df_articulos = pd.read_sql(articulos_query, engine)
df_articulos.info()

In [ ]:
# Agrupamos por proveedor.
df_grouped_ref = df_articulos[['AM_COD_PROVEEDOR','PR_NOMBRE_PROVEEDOR','AM_PN']]\
                .groupby(['AM_COD_PROVEEDOR','PR_NOMBRE_PROVEEDOR'],as_index=False)\
                .agg('count')
# Las referencias a 1?????, fallos en importacion.
df_grouped_ref   

In [ ]:
# Agrupamos por proveedor. Aquellos que tengan stock.
df_grouped_stock = df_articulos[df_articulos['AM_STOCK']>0][['AM_COD_PROVEEDOR','PR_NOMBRE_PROVEEDOR','AM_PN']]\
            .groupby(['AM_COD_PROVEEDOR','PR_NOMBRE_PROVEEDOR'],as_index=False)\
            .agg('count')
df_grouped_ref

In [ ]:
df_no_importados = df_grouped_ref[df_grouped_ref['AM_PN']==1]
df_no_importados = df_no_importados.drop(columns=['AM_PN'])
df_no_importados.to_csv(os.getcwd()+r'\no_importados_{}.csv'.format(time.strftime("%d-%m-%y")), sep=';', decimal=',',index=False)
df_no_importados

In [ ]:
# Cruzamos las tablas.
df_referencia_stock = pd.merge(df_grouped_ref, df_grouped_stock, on=['AM_COD_PROVEEDOR'])
df_referencia_stock = df_referencia_stock.drop(columns=['PR_NOMBRE_PROVEEDOR_y'])
df_referencia_stock.columns = ['CODIGO', 'PROVEEDOR', 'REFERENCIAS', 'CON STOCK']
df_referencia_stock_sort = df_referencia_stock.sort_values (['REFERENCIAS'], ascending=False)
df_referencia_stock_sort

In [ ]:
df = df_referencia_stock
df['PORCENTAJE CON STOCK'] = round((df['CON STOCK']*100)/df['REFERENCIAS'],2)
df = df.sort_values (['PORCENTAJE CON STOCK'], ascending=False)
df.to_csv(os.getcwd()+r'\proveedor_con_stock_{}.csv'.format(time.strftime("%d-%m-%y")), sep=';', decimal=',',index=False)
df

In [ ]:
# Porcentaje medio con stock
df['PORCENTAJE CON STOCK'].mean()

In [ ]:
# Porcentaje medio con stock sin INGRAM MICRO
df = df[df.CODIGO != '000048']
df['PORCENTAJE CON STOCK'].mean()

In [ ]:
# Calculos de procentajes de stock por fabricantes.
df_grouped_fab = df_articulos[['AM_COD_PROVEEDOR','PR_NOMBRE_PROVEEDOR','AM_FABRICANTE','f_nombre_fabricante','AM_PN']]\
                .groupby(['AM_FABRICANTE','f_nombre_fabricante','AM_COD_PROVEEDOR','PR_NOMBRE_PROVEEDOR'],as_index=False)\
                .agg('count')
df_grouped_fab = df_grouped_fab.sort_values(['AM_FABRICANTE','AM_PN'], ascending=[True, False])
df_grouped_fab

In [ ]:
df_grouped_fab_stock = df_articulos[df_articulos['AM_STOCK']>0]\
                [['AM_COD_PROVEEDOR','PR_NOMBRE_PROVEEDOR','AM_FABRICANTE','f_nombre_fabricante','AM_PN']]\
                .groupby(['AM_FABRICANTE','f_nombre_fabricante','AM_COD_PROVEEDOR','PR_NOMBRE_PROVEEDOR'],as_index=False)\
                .agg('count')
df_grouped_fab_stock = df_grouped_fab_stock.sort_values(['AM_FABRICANTE','AM_PN'], ascending=[True, False])
df_grouped_fab_stock

In [ ]:
df_referencia_stock = pd.merge(df_grouped_fab, df_grouped_fab_stock, on=['AM_FABRICANTE','AM_COD_PROVEEDOR'])
df_referencia_stock = df_referencia_stock.drop(columns=['f_nombre_fabricante_y','PR_NOMBRE_PROVEEDOR_y'])
df_referencia_stock.columns = ['CODIGO_FA','FABRICANTE','CODIGO_PR','PROVEEDOR','REFERENCIAS','CON STOCK']
df_referencia_stock

In [ ]:
df = df_referencia_stock
df['PORCENTAJE CON STOCK'] = round((df['CON STOCK']*100)/df['REFERENCIAS'],2)
df = df.sort_values (['CODIGO_FA','PORCENTAJE CON STOCK'], ascending=[True, False])
df.to_csv(os.getcwd()+r'\fabricantes_proveedor_con_stock_{}.csv'.format(time.strftime("%d-%m-%y")),\
            sep=';',decimal=',',index=False)
df

In [ ]:
# Calculos de procentajes de stock por fabricante/subfamilia.
df_grouped_subf = df_articulos[['AM_SUBFAMILIA','s_nombre_subfamilia','AM_COD_PROVEEDOR','PR_NOMBRE_PROVEEDOR','AM_PN']]\
                .groupby(['AM_SUBFAMILIA','s_nombre_subfamilia','AM_COD_PROVEEDOR','PR_NOMBRE_PROVEEDOR'], as_index=False)\
                .agg('count')
        
df_grouped_subf = df_grouped_subf.sort_values(['AM_SUBFAMILIA','AM_PN'], ascending=[True, False])
df_grouped_subf

In [ ]:
df_grouped_subf_stock = df_articulos[df_articulos['AM_STOCK']>0]\
               [['AM_SUBFAMILIA','s_nombre_subfamilia','AM_COD_PROVEEDOR','PR_NOMBRE_PROVEEDOR','AM_PN']]\
                .groupby(['AM_SUBFAMILIA','s_nombre_subfamilia','AM_COD_PROVEEDOR','PR_NOMBRE_PROVEEDOR'], as_index=False)\
                .agg('count')
df_grouped_subf_stock = df_grouped_subf_stock.sort_values(['AM_SUBFAMILIA','AM_PN'], ascending=[True, False])
df_grouped_subf_stock

In [ ]:
df_referencia_stock = pd.merge(df_grouped_subf, df_grouped_subf_stock, on=['AM_SUBFAMILIA','AM_COD_PROVEEDOR'])
df_referencia_stock = df_referencia_stock.drop(columns=['s_nombre_subfamilia_y','PR_NOMBRE_PROVEEDOR_y'])
df_referencia_stock.columns = ['CODIGO_SU','SUBFAMILIA','CODIGO_PR','PROVEEDOR','REFERENCIAS','CON STOCK']
df_referencia_stock

In [ ]:
df = df_referencia_stock
df['PORCENTAJE CON STOCK'] = round((df['CON STOCK']*100)/df['REFERENCIAS'],2)
df = df.sort_values (['CODIGO_SU','PORCENTAJE CON STOCK'], ascending=[True, False])
df.to_csv(os.getcwd()+r'\subfamilias_proveedor_con_stock_{}.csv'.format(time.strftime("%d-%m-%y")),\
            sep=';',decimal=',',index=False)
df

In [ ]:
# Operativas sobre el detalle de un fabricante concreto
fabricantes_query = open(os.getcwd()+r'\fabricantes.sql').read()
df_fabricantes = pd.read_sql(fabricantes_query, engine)
df_fabricantes.to_csv(os.getcwd()+r'\fabricantes_{}.csv'.format(time.strftime("%d-%m-%y")),\
            sep=';',decimal=',',index=False)

In [ ]:
fabricante = ''
proveedor = ''
df = df_articulos[(df_articulos['f_nombre_fabricante'] == fabricante) & (df_articulos['PR_NOMBRE_PROVEEDOR'] == proveedor)]
df = df.drop(columns=['AM_PVP','AM_FABRICANTE','f_nombre_fabricante'])
df = df.sort_values(['AM_SUBFAMILIA','AM_STOCK'], ascending = [True, False])
df.to_csv(os.getcwd()+r'\fabricante_{}_proveedor_{}_detalle{}.csv'.format(fabricante,proveedor,time.strftime("%d-%m-%y")),\
            sep=';', encoding='iso-8859-1', index=False)
df